In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

from utils.corpus import *

# Building corpus file
Processed files have a '_processed' postfix before extension

In [2]:
build_corpus('data/corpus/defthomes_musevana.txt')
data = open('data/corpus/defthomes_musevana_processed.txt').read()

In [3]:
tokenizer = Tokenizer()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{'you': 1, 'i': 2, 'the': 3, 'and': 4, 'me': 5, 'a': 6, 'to': 7, 'it': 8, 'in': 9, 'your': 10, 'my': 11, "i'm": 12, 'of': 13, "don't": 14, 'is': 15, 'all': 16, 'no': 17, 'we': 18, 'on': 19, 'be': 20, 'that': 21, 'have': 22, 'up': 23, 'so': 24, 'for': 25, 'out': 26, 'this': 27, "it's": 28, 'with': 29, 'but': 30, "you're": 31, 'not': 32, 'just': 33, 'will': 34, 'yeah': 35, 'can': 36, 'what': 37, 'now': 38, 'when': 39, 'she': 40, 'are': 41, 'do': 42, 'like': 43, 'know': 44, 'take': 45, 'get': 46, 'away': 47, 'want': 48, 'could': 49, 'one': 50, "i'll": 51, "can't": 52, 'down': 53, 'if': 54, 'oh': 55, 'never': 56, 'love': 57, 'come': 58, 'let': 59, 'go': 60, 'from': 61, 'feel': 62, 'way': 63, 'time': 64, 'they': 65, 'been': 66, 'think': 67, 'back': 68, 'got': 69, 'should': 70, 'gonna': 71, 'was': 72, 'our': 73, 'at': 74, 'see': 75, 'us': 76, 'as': 77, 'again': 78, 'more': 79, 'here': 80, 'home': 81, 'make': 82, 'there': 83, 'into': 84, 'why': 85, 'too': 86, 'care': 87, 'hey': 88, "won't": 8

In [4]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
print(model)

Epoch 1/100
 182/4649 [>.............................] - ETA: 20:21 - loss: 8.1442 - accuracy: 0.0166